In [1]:
import creds
from googleapiclient.discovery import build
import pandas as pd
import seaborn as sns

In [16]:
API_KEY = creds.YT_api
channel_ids = ['UCX6OQ3DkcsbYNE6H8uQQuVA',
               'UCxgAuX3XZROujMmGphN_scA',
               'UCqECaJ8Gagnn7YCbPEzWH6g',
               'UC0WP5P-ufpRfjbNrmOWwLBQ']
influencer = 'Mark Tilbury'


In [24]:
api_service_name = "youtube"
api_version = "v3"
client_secrets_file = "YOUR_CLIENT_SECRET_FILE.json"
youtube = build(api_service_name, api_version, developerKey=API_KEY)


# Function to get channel stats for further analysis for the Vibecheck

In [25]:
def get_channel_stats(youtube, channel_ids):
    data_list = []
    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=','.join(channel_ids)
    )
    response = request.execute()
    for i in range(len(response['items'])):
        data = dict(channel_name = response['items'][i]['snippet']['title'],
                    subscriber_count = response['items'][i]['statistics']['subscriberCount'],
                    view_count = response['items'][i]['statistics']['viewCount'],
                    video_count = response['items'][i]['statistics']['videoCount'],
                    playlist_id = response['items'][i]['contentDetails']['relatedPlaylists']['uploads'])
        data_list.append(data)
    return data_list

In [26]:
channel_data = get_channel_stats(youtube, channel_ids)

In [32]:
# Function to get the latest video link from a playlist
def get_latest_video_link(youtube, playlist_id):
    request = youtube.playlistItems().list(
        part="snippet",
        playlistId=playlist_id,
        maxResults=1
    )
    response = request.execute()
    video_id = response['items'][0]['snippet']['resourceId']['videoId']
    video_link = f"https://www.youtube.com/watch?v={video_id}"
    return video_link

In [33]:
# Add latest video link to each channel's data
for channel in channel_data:
    latest_video_link = get_latest_video_link(youtube, channel['playlist_id'])
    channel['latest_video_link'] = latest_video_link

In [ ]:
stats = pd.DataFrame(channel_data)

In [35]:
stats['subscriber_count'] = pd.to_numeric(stats['subscriber_count'])
stats['view_count'] = pd.to_numeric(stats['view_count'])
stats['video_count'] = pd.to_numeric(stats['video_count'])

In [36]:
stats.to_csv('stats.csv')

In [37]:
stats

,channel_name,subscriber_count,view_count,video_count,playlist_id,latest_video_link
0,Taylor Swift,60100000,36912943387,274,UUqECaJ8Gagnn7YCbPEzWH6g,https://www.youtube.com/watch?v=b6zPqm0lLQs
1,The Weeknd,36200000,28856231678,188,UU0WP5P-ufpRfjbNrmOWwLBQ,https://www.youtube.com/watch?v=UhNwFRzJ1jw
2,Mark Tilbury,3730000,615573351,164,UUxgAuX3XZROujMmGphN_scA,https://www.youtube.com/watch?v=PjH5DJkR0k4
3,MrBeast,319000000,60450070160,820,UUX6OQ3DkcsbYNE6H8uQQuVA,https://www.youtube.com/watch?v=Kt2HvqRruHQ
